## NYCHA Property Directory Block and Lot Guide Parser
**Developed by**: Itzamna Huerta, for the Association for Neighborhood and Housing Development (ANHD)  
**Created**: September 2024  
**Last Updated**: N/A  

---

### Overview:

This Python script extraacts and processes data from the NYCHA Property Directory Block and Lot Guide PDF, making it more clear for internal purposes at ANHD [JustFixNYC’s NYCHA Scraper](https://github.com/JustFixNYC/nycha-scraper/tree/master). Using **pdfplumber** for table extraction and **pandas** for organizing data, the script cleans and structures PDF data for easier analysis. It resolves issues like irregular row lengths and missing information. 

The script’s primary functionality includes handling complex multi-page tables, automatically tagging rows with the correct **borough**, and identifying "bad" rows (those that don't meet the expected structure), ensuring accurate borough data association across the entire dataset.

---

### Key Features:

- **Enhanced Data Processing**: Improves JustFixNYC’s JavaScript approach by handling more complex data structures and errors.
- **Error Handling**: Separates and cleans rows that don’t match the expected format.
- **Borough Categorization**: Ensures each row is labeled with its borough.

---

### Methodology:
1. **Borough Detection**: Scans each page to identify its borough and appends it to the data.
2. **PDF Processing**: Extracts and separates valid and invalid rows based on their structure (8 columns expected).
3. **Data Cleaning**: Cleans valid rows by removing unnecessary data and formatting errors, while also cleaning invalid rows for potential use.
4. **Final Output**: Merges cleaned data and exports it to a CSV for further analysis.


Final Output: Merges cleaned data and exports it to a CSV for further analysis.
---

### Example Usage:

```python
# Process the PDF and obtain the dataframes
combined_df, bad_rows_df = process_pdf_to_dataframe('./2024/Block-and-Lot-Guide-2024.pdf')

# Export to CSV for further analysis
combined_df.to_csv('NYCHA_Valid_Data.csv', index=False)
bad_rows_df.to_csv('NYCHA_Bad_Rows.csv', index=False)


In [1]:
# Import Libraries
import pdfplumber
import pandas as pd
import re

In [2]:
# Filepath name
pdf_path = "./2024/Block-and-Lot-Guide-2024.pdf"

In [3]:
# Define Functions and Process the PDF
def extract_borough_from_text(full_text):
    '''Extracts the borough name from the full text of a PDF page.'''
    boroughs = ["MANHATTAN", "BROOKLYN", "QUEENS", "BRONX", "STATEN ISLAND"]
    for borough in boroughs:
        if borough in full_text:
            return borough
    return None

def process_pdf_to_dataframe(pdf_path):
    '''Processes the PDF file and returns a combined DataFrame with borough info and a DataFrame of bad rows.'''
    all_tables_df = []
    all_bad_rows = []

    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)
        print(f"Total number of pages: {total_pages}")

        # Process each page of the PDF
        for i, page in enumerate(pdf.pages):
            # You can uncomment to see the progress of each page
            # print(f"Processing page {i+1} of {total_pages}")
            
            # Extract borough from the full text
            full_text = page.extract_text()
            borough = extract_borough_from_text(full_text)

            if borough is None:
                print(f"No borough found on page {i+1}. Skipping.")
                continue  # Skip if no borough is found

            # Extract tables from the page
            tables = page.extract_tables()

            # Process each table on the page
            for table_index, table in enumerate(tables):
                # Skip empty tables
                if not table:
                    print(f"Table {table_index + 1} on page {i+1} is empty after processing.")
                    continue

                # Append borough information to bad rows if any row doesn't match the expected length
                for row in table:
                    if len(row) != 8:  # Expected length is 8, adjust if different
                        row.append(borough)  # Add the borough to the row as the last column
                        all_bad_rows.append(row)  # Collect bad rows

                # Check if valid table rows (i.e., with expected column length)
                valid_rows = [row for row in table if len(row) == 8]
                if valid_rows:
                    # Append borough information to each valid row
                    for row in valid_rows:
                        row.append(borough)  # Add borough to valid rows
                    
                    # Create a DataFrame from the valid rows
                    df = pd.DataFrame(valid_rows)
                    all_tables_df.append(df)

    # Combine all valid DataFrames into a single DataFrame
    if all_tables_df:
        combined_df = pd.concat(all_tables_df, ignore_index=True)
    else:
        combined_df = pd.DataFrame()

    # Create a DataFrame for bad rows
    if all_bad_rows:
        # Create DataFrame from bad rows, ignoring column names and just adding the borough at the end
        bad_rows_df = pd.DataFrame(all_bad_rows)
    else:
        bad_rows_df = pd.DataFrame()

    return combined_df, bad_rows_df

# Run the function
combined_df, bad_rows_df = process_pdf_to_dataframe(pdf_path)


Total number of pages: 85


In [4]:
#####
# Additional data cleaning for combined_df
#####

# Set the first row as the header
combined_df.columns = combined_df.iloc[0]

# Drop the first row (now the header)
combined_df = combined_df[1:].reset_index(drop=True)

# Update header name from BRONX to BOROUGH 
combined_df.rename(columns = {'BRONX':'BOROUGH'}, inplace = True)

# Replace all instances of \n within any column in the dataframe
combined_df.replace('\n', ' ', regex=True, inplace=True)

# There are 8 rows where this condition is met. We are going to update the dataset by removing those rows.
combined_df = combined_df[~combined_df['BLOCK'].str.contains('BLOCK', na=False)]

# Remove rows where 'ADDRESS' is empty, these could be parking lots, air rights, etc. 
combined_df = combined_df[combined_df['ADDRESS'] != '']

# Convert 'BLOCK' to int64, replacing non-convertible cells with 0
combined_df['BLOCK'] = pd.to_numeric(combined_df['BLOCK'].str.strip(), errors='coerce').fillna(0).astype(int)

# Drop rows where 'BLOCK' contains 0
combined_df = combined_df[combined_df['BLOCK'] != 0]

# Remove rows where 'ZIP CODE' is empty, it's either vacant land, parking lot, development grounds, etc.
combined_df = combined_df[combined_df['ZIP CODE'] != '']

# This approach first replaces empty strings with 0, converts the column to float, fills any NaN values, and finally converts to int.
combined_df['CD#'] = pd.to_numeric(combined_df['CD#'].replace('', '0'), errors='coerce').fillna(0).astype(int)

# Remove CD's that equal 0, these 2 records are development grounds
combined_df = combined_df[combined_df['CD#'] != 0]

# Update following columns to int
combined_df[['LOT', 'ZIP CODE', 'CD#']] = combined_df[['LOT', 'ZIP CODE', 'CD#']].astype(int)

In [5]:
combined_df.dtypes

0
BLOCK           int64
LOT             int64
ADDRESS        object
ZIP CODE        int64
DEVELOPMENT    object
MANAGED BY     object
CD#             int64
FACILITY       object
BOROUGH        object
dtype: object

In [6]:
combined_df.head()

,BLOCK,LOT,ADDRESS,ZIP CODE,DEVELOPMENT,MANAGED BY,CD#,FACILITY,BOROUGH
0,2215,116,5210 BROADWAY,10463,MARBLE HILL,MARBLE HILL,8,ROOFTOP LEASE LOCATION,BRONX
1,2215,116,5220 BROADWAY,10463,MARBLE HILL,MARBLE HILL,8,DEVELOPMENT MANAGEMENT OFFICE,BRONX
2,2215,116,5470 BROADWAY,10463,MARBLE HILL,MARBLE HILL,8,CHILDREN CENTER,BRONX
3,2215,116,5480 BROADWAY,10463,MARBLE HILL,MARBLE HILL,8,NURSERY SCHOOL,BRONX
4,2215,116,5360 BROADWAY,10463,MARBLE HILL,MARBLE HILL,8,,BRONX


In [7]:
# Further data cleaning for the bad rows in order to merge it with the combined_df 

pd.options.display.max_rows=2000

# Rename the columns
bad_rows_df.rename(columns={ 0: 'BLOCK', 
                             1: 'LOT', 
                             2: 'ADDRESS', 
                             3: 'TO_BE_DELETED_1',
                             4: 'ZIP CODE',
                             5: 'DEVELOPMENT',
                             6: 'TO_BE_DELETED_2',
                             7: 'MANAGED BY',
                             8: 'CD#',
                             9: 'FACILITY',
                             10: 'BOROUGH'
                            }, inplace=True)

bad_rows_df.drop(columns=['TO_BE_DELETED_1', 'TO_BE_DELETED_2'], inplace=True)

# header_bad_rows = bad_rows_df[bad_rows_df['BLOCK'].str.contains('BLOCK', na=False)]
# header_bad_rows.shape
# Result: (76, 9), plus the 8 found in the combined_df is 84 which is expected

# There are 76 rows where this condition is met. We are going to update the dataset by removing those rows.
bad_rows_df = bad_rows_df[~bad_rows_df['BLOCK'].str.contains('BLOCK', na=False)]

# Replace all instances of \n within any column in the dataframe
bad_rows_df.replace('\n', ' ', regex=True, inplace=True)

# Convert 'BLOCK' to int64, replacing non-convertible cells with 0
bad_rows_df['BLOCK'] = pd.to_numeric(bad_rows_df['BLOCK'].str.strip(), errors='coerce').fillna(0).astype(int)

# Drop rows where 'BLOCK' contains 0
bad_rows_df = bad_rows_df[bad_rows_df['BLOCK'] != 0]

# Remove rows where 'ZIP CODE' is empty, it's either vacant land, parking lot, development grounds, etc.
bad_rows_df = bad_rows_df[bad_rows_df['ZIP CODE'] != '']

# This approach first replaces empty strings with 0, converts the column to float, fills any NaN values, and finally converts to int.
bad_rows_df['CD#'] = pd.to_numeric(bad_rows_df['CD#'].replace('', '0'), errors='coerce').fillna(0).astype(int)

# Remove CD's that equal 0, these 2 records are development grounds
bad_rows_df = bad_rows_df[bad_rows_df['CD#'] != 0]

# Update following columns to int
bad_rows_df[['LOT', 'ZIP CODE', 'CD#']] = bad_rows_df[['LOT', 'ZIP CODE', 'CD#']].astype(int)


In [8]:
bad_rows_df.dtypes

BLOCK           int64
LOT             int64
ADDRESS        object
ZIP CODE        int64
DEVELOPMENT    object
MANAGED BY     object
CD#             int64
FACILITY       object
BOROUGH        object
dtype: object

In [9]:
bad_rows_df.shape

(1009, 9)

In [10]:
# Merge the bad batch with combined_df 
merged_df = pd.concat([combined_df, bad_rows_df], ignore_index=True, sort=False)

In [11]:
merged_df.dtypes

BLOCK           int64
LOT             int64
ADDRESS        object
ZIP CODE        int64
DEVELOPMENT    object
MANAGED BY     object
CD#             int64
FACILITY       object
BOROUGH        object
dtype: object

In [12]:
# Desired order of columns
desired_order = ['BOROUGH', 'BLOCK', 'LOT', 'ADDRESS', 'ZIP CODE', 'DEVELOPMENT', 'MANAGED BY', 'CD#', 'FACILITY']

# Rearranging the columns
merged_df = merged_df[desired_order]

In [13]:
# Save merged dataset to a csv for further comparison 
merged_df.to_csv('./2024/Block-and-Lot-Guide-01012024-ANHD.csv', index=False)